In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

# Go up one directory from `b/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()

DEFAULT_TRANSFORMERS_MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 16_000,
    "load_in_4bit": True,
    "enable_thinking": False,  # main_gpu interpretation
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **DEFAULT_TRANSFORMERS_MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║ ▓▓▓▓▓   ▓▓▓▓                                  ▓▓▓                ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓   ▓▓▓▓ ▓ ▓       ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓  ▓▁▁▓ ▓▓        ║
║  ▓▓   ▓▓ ▓▓  ▓    ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓   ▓    ▓         ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓   ▓▓▓▓ ▓         ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready ║
║ Faster builds, clearer flows, production-first                   ║
║                                                                  ║
║ Version: 0.1.3 | Python: 3.11.13                                 ║
║ OS: Linux 6.14.0-33-generic (x86_64)                             ║
║ Torch: 2.7.1+cu126   CUDA: yes (12.6)                            ║
║ MPS: no (built: False)          

/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     | 2025-11-06 11:07:57 | config.py:<module> | PyTorch version 2.7.1+cu126 available.


/home/nomi/anaconda3/envs/LLMs/lib/python3.11/importlib/__init__.py:126: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-06 11:07:58 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-11-06 11:08:00 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-11-06 11:08:00 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.
INFO     | 2025-11-06 11:08:01 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.92s/it]

INFO     | 2025-11-06 11:08:07 | transformers.py:init_model | Transformers model initialized successfully.


In [3]:
from IPython.display import display, Markdown, clear_output

system_prompt = "You are a joker."
user_prompt = """Tell me a light-hearted joke."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    print(chunk, flush=True, end="")

Why don't skeletons fight each other?  
Because they don't have the *guts*! 😄

## RAG wiring so the agent “understands” the Neurosurf codebase

You’ll ingest the repo once, then run a retriever to answer code questions. The Planner can call the retriever first to form a precise implementation plan.

### FileReader and Chunker Test

In [4]:
# scripts/index_repo_for_rag.py
from pathlib import Path
from neurosurfer.rag.ingestor import RAGIngestor
from neurosurfer.rag.chunker import Chunker
from neurosurfer.rag.filereader import FileReader
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder

embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
vs = ChromaVectorStore(collection_name="neurosurf-repo")
ing = RAGIngestor(
    embedder=embedder,
    vector_store=vs, 
    chunker=Chunker(), 
    file_reader=FileReader(),
    default_metadata={"collection": "neurosurf"}
)

root_dir = Path(os.getcwd()).parent.joinpath("neurosurfer")
ing.add_directory(root_dir)  # the repo root
print(ing.build())


INFO     | 2025-11-06 11:08:09 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-06 11:08:09 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-11-06 11:08:09 | posthog.py:__init__ | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[Init] ChromaVectorStore initialized with collection: neurosurf-repo


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.03it/s]


{'status': 'ok', 'sources': 99, 'chunks': 608, 'unique_chunks': 608, 'added': 608, 'finished_at': 1762412892.396136}


## RAG AGENT

In [5]:
# scripts/rag_helper.py
from neurosurfer.agents.rag import RAGAgent, RAGAgentConfig
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
from neurosurfer.models.chat_models.openai import OpenAIModel

vs = ChromaVectorStore(collection_name="neurosurf-repo")
# embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
# llm = OpenAIModel(model_name="gpt-4o-mini")
rag_agent = RAGAgent(
    llm=LLM, 
    vectorstore=vs, 
    embedder=embedder, 
    config=RAGAgentConfig(top_k=6)
)

[Init] ChromaVectorStore initialized with collection: neurosurf-repo


Test RagAgent

In [6]:
from IPython.display import display, Markdown

# test retrieve
# retrieved_obj = rag_agent.retrieve("What is the main difference between transformers and unsloth?")
# print(retrieved_obj.context)

# test run
llm_response = rag_agent.run("Explain Chunker Configurations in table form.")
response = ""
md_display = display(Markdown(""), display_id=True)
for chunk in llm_response:
    response += chunk
    md_display.update(Markdown(response))


Here's a **table summarizing the key configurations of the `Chunker` class** based on the provided code and documentation:

| **Configuration Parameter**         | **Description**                                                                 | **Default Value**         | **Notes**                                                                 |
|------------------------------------|---------------------------------------------------------------------------------|---------------------------|---------------------------------------------------------------------------|
| `fallback_chunk_size`             | Maximum number of lines for fallback chunking (used when no specific strategy is found). | `30`                      | Applies to line-based chunking if no other strategy is available.       |
| `overlap_lines`                   | Number of lines to overlap between consecutive chunks to preserve context.       | `5`                       | Helps maintain continuity between chunks.                               |
| `char_chunk_size`                 | Maximum number of characters per chunk for character-based chunking.            | `1000`                    | Used for prose or generic text content.                                  |
| `comment_block_threshold`         | Minimum number of comment lines to consider a block as comment-only.            | `4`                       | Used to filter out comment-only blocks from chunks.                      |
| `blacklist_patterns`              | List of regex patterns for files to exclude (e.g., `.git`, `node_modules`, etc.). | As defined in code         | Excludes unwanted file types (e.g., binaries, config files, etc.).       |
| `line_comment_markers`            | List of comment markers for line-based comment detection (e.g., `#`, `//`).      | `["#", "//", "/*", "*/"]`  | Used to identify and filter comment-only lines.                           |
| `strategy_priority`               | Priority order for selecting chunking strategies (e.g., custom, router, etc.).   | Defined in code           | Priority order: 1. Custom handler, 2. Router, 3. File extension mapping, 4. Built-in strategy, 5. Heuristic fallback. |
| `chunking_strategy`               | The main chunking strategy to use (e.g., line-based, character-based, AST-based).| Determined dynamically     | Selected based on file type, content, and configuration.                |
| `safety_limits`                   | Safety limits for chunking (e.g., maximum chunk size, minimum content length).   | Defined in `ChunkerConfig`| Prevents overly large or small chunks.                                   |
| `custom_handlers`                 | Dictionary of custom handlers for specific file extensions or content types.     | Empty by default          | Registered via `register()` method.                                      |
| `router_function`                 | A function to dynamically select a chunking strategy based on content analysis.  | Not set by default        | Used for advanced strategy selection.                                    |
| `file_extension_mapping`         | Mapping of file extensions to specific chunking strategies.                      | Defined in code           | Used to override default strategies for specific file types.             |

---

### Summary:
This table provides a structured overview of the **key configuration parameters** of the `Chunker` class, including their **purpose**, **default values**, and **notes**. These configurations allow for **flexible and intelligent chunking** of documents in a RAG (Retrieval-Augmented Generation) system.

Batches: 100%|██████████| 1/1 [00:00<00:00, 29.36it/s]

INFO     | 2025-11-06 11:08:12 | agent.py:retrieve | [RAGRetriever] Retrieved 6 chunks


## Tools Router Agent Test

In [7]:
from neurosurfer.agents.tools_router_agent import ToolsRouterAgent
from neurosurfer.tools.toolkit import Toolkit

from neurosurfer.tools.tool_spec import ToolSpec, ToolParam, ToolReturn
from neurosurfer.tools.base_tool import BaseTool, ToolResponse
from neurosurfer.tools.common.general_query_assistant import GeneralQueryAssistantTool

# Simple Calculator Tool
class CalculatorTool(BaseTool):
    spec = ToolSpec(
        name="calculator",
        description="Perform basic arithmetic operations such as addition, subtraction, multiplication, and division.",
        when_to_use="Use this tool when you need to perform basic arithmetic operations.",
        inputs=[
            ToolParam(name="num1", type="float", description="The first number.", required=True),
            ToolParam(name="num2", type="float", description="The second number.", required=True),
            ToolParam(name="operation", type="string", description="The operation to perform: 'add', 'subtract', 'multiply', or 'divide'.", required=True)
        ],
        returns=ToolReturn(type="float", description="The result of the arithmetic operation.")
    )

    def __init__(self, final_answer: bool = False):
        self.final_answer = final_answer

    def __call__(self, num1: float, num2: float, operation: str) -> ToolResponse:
        if operation not in ["add", "subtract", "multiply", "divide"]:
            return ToolResponse(
                final_answer=False,
                observation="Invalid operation. Supported operations are 'add', 'subtract', 'multiply', and 'divide'.",
                extras={}
            )
        
        if operation == "divide" and num2 == 0:
            return ToolResponse(
                final_answer=False,
                observation="Division by zero is not allowed.",
                extras={}
            )
        try:
            num1 = float(num1)
            num2 = float(num2)
            if operation == "add":
                result = num1 + num2
            elif operation == "subtract":
                result = num1 - num2
            elif operation == "multiply":
                result = num1 * num2
            elif operation == "divide":
                result = num1 / num2
        except Exception as e:
            return ToolResponse(
                final_answer=False,
                observation=f"An error occurred: {str(e)}",
                extras={}
            )
        
        return ToolResponse(
            final_answer=self.final_answer,
            observation=float(result),
            extras={}
        )

toolkit = Toolkit(
    tools=[
        CalculatorTool(),
        GeneralQueryAssistantTool(llm=LLM, stream=True)
    ]
)

tools_router_agent = ToolsRouterAgent(
    toolkit=toolkit,
    llm=LLM,
    verbose=True,
    specific_instructions=""
)


INFO     | 2025-11-06 11:08:31 | toolkit.py:register_tool | Registered tool: calculator
INFO     | 2025-11-06 11:08:31 | toolkit.py:register_tool | Registered tool: general_query_assistant


Test ToolsRouterAgent

In [8]:
query = "Perform the calculation 20 * 90"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")


INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Using tool: calculator
INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Raw inputs: {'num1': 20.0, 'num2': 90.0, 'operation': 'multiply'}
1800.0

In [9]:
query = "Tell me a light-hearted joke!"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")

INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Using tool: general_query_assistant
INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Raw inputs: {'query': 'Tell me a light-hearted joke!'}
Why don't skeletons fight each other? They don't have the guts!None

## ReactAgent

In [ ]:
from neurosurfer.agents.react import ReActAgent, ReActConfig

react_agent = ReActAgent(
    toolkit=toolkit,
    llm=LLM,
    specific_instructions="Always be concise in your answers. Break the task into steps if needed.",
    config=ReActConfig(
        temperature=0.7,
        max_new_tokens=4096,
        allow_input_pruning=True,
        repair_with_llm=True,
        skip_special_tokens=True,
        verbose=True
    )
)

# print(react_agent._system_prompt())
TASK = """Calculate 100 + 200 / 300. Then tell me a light-hearted joke about that result."""

for chunk in react_agent.run(TASK):
    print(chunk, end="")




[🧠] Chain of Thoughts...
Thought: I need to calculate 100 + 200 / 300 first, and then find a light-hearted joke based on the result. Let's start with the calculation.

Action: {
  "tool": "calculator",
  "inputs": {
    "num1": 100,
    "num2": 200 / 300,
    "operation": "add"
  },
  "final_answer": false
}<__final_answer__>The result of $100 + \frac{200}{300}$ is approximately $100.67$. 

Here's a light-hearted joke:  
Why did the fraction get a promotion?  
Because it was *always* in its **best interest**! 😄</__final_answer__>